# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Utility-Functions" data-toc-modified-id="Utility-Functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Utility Functions</a></div><div class="lev1 toc-item"><a href="#MNIST" data-toc-modified-id="MNIST-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>MNIST</a></div><div class="lev2 toc-item"><a href="#Deep-Autoencoder" data-toc-modified-id="Deep-Autoencoder-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Deep Autoencoder</a></div><div class="lev2 toc-item"><a href="#Shallow-Autoencoder" data-toc-modified-id="Shallow-Autoencoder-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Shallow Autoencoder</a></div><div class="lev1 toc-item"><a href="#Denoising-Autoencoder" data-toc-modified-id="Denoising-Autoencoder-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Denoising Autoencoder</a></div><div class="lev1 toc-item"><a href="#Sparse-Autoencoders" data-toc-modified-id="Sparse-Autoencoders-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Sparse Autoencoders</a></div>

```
Author: Roy Wilds
Created: 2020-12-12
Description: Combining various sources (see credits below) to provide a complete example of using an Autoencoder with Keras (with Tensorflow) that covers the "gotchas" that I found challenging.
```

Credits:
- Relies heavily on the great work from https://github.com/ardendertat/Applied-Deep-Learning-with-Keras/blob/master/notebooks/Part%203%20-%20Autoencoders.ipynb 
- Incorporates material from E2EML Course 312: https://end-to-end-machine-learning.teachable.com/courses/enrolled/612528

Layout of this notebook is
- Import what's needed and then define some handy functions.
- Create our dataset
- Learn a simple 5 layer model and then understand what it's doing.
- Learn a sparse 3 layer model (just 1 hidden layer). Compare with a non-sparse model of the same dimensions.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

from __future__ import print_function
from keras.models import Model
from keras.layers import Dense, Input
from keras.datasets import mnist
from keras.regularizers import l1
from keras.optimizers import Adam

# Utility Functions
Small modifications from original: https://github.com/ardendertat/Applied-Deep-Learning-with-Keras/blob/master/notebooks/Part%203%20-%20Autoencoders.ipynb

Keep in mind that these functions rely on having access to some global variables (e.g. `x_test`)

In [ ]:
def plot_autoencoder_outputs(autoencoder, dims, n=5):
    decoded_imgs = autoencoder.predict(x_test)
    plt.figure(figsize=(10, 4.5))
    for i in range(n):
        # plot original image
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(x_test[i].reshape(*dims))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        if i == n/2:
            ax.set_title('Original Images')

        # plot reconstruction 
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(decoded_imgs[i].reshape(*dims))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        if i == n/2:
            ax.set_title('Reconstructed Images')
    plt.show()

def plot_loss(history):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    plt.figure(figsize=(8, 6))
    historydf.plot(ylim=(0, historydf.values.max()))
    plt.title('Loss: %.3f' % history.history['loss'][-1])
    
def plot_compare_histories(history_list, name_list, plot_accuracy=True):
    dflist = []
    min_epoch = len(history_list[0].epoch)
    losses = []
    for history in history_list:
        h = {key: val for key, val in history.history.items() if not key.startswith('val_')}
        dflist.append(pd.DataFrame(h, index=history.epoch))
        min_epoch = min(min_epoch, len(history.epoch))
        losses.append(h['loss'][-1])

    historydf = pd.concat(dflist, axis=1)

    metrics = dflist[0].columns
    idx = pd.MultiIndex.from_product([name_list, metrics], names=['model', 'metric'])
    historydf.columns = idx
    
    plt.figure(figsize=(6, 8))

    ax = plt.subplot(211)
    historydf.xs('loss', axis=1, level='metric').plot(ylim=(0,1), ax=ax)
    plt.title("Training Loss: " + ' vs '.join([str(round(x, 3)) for x in losses]))
    
    if plot_accuracy:
        ax = plt.subplot(212)
        historydf.xs('acc', axis=1, level='metric').plot(ylim=(0,1), ax=ax)
        plt.title("Accuracy")
        plt.xlabel("Epochs")
    
    plt.xlim(0, min_epoch-1)
    plt.tight_layout()

# Data
Sticking with Arden Dertat's article/code, we'll use the MNIST dataset. https://towardsdatascience.com/applied-deep-learning-part-3-autoencoders-1c083af4d798

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize!
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
# Reshape so that we have a flat vector for each image
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

print(x_train.shape)
print(x_test.shape)

# Simple Autoencoder Structure
Define our layer sizes. We have:
- Input Layer (layer size is determined by image size)
- Encoder Hidden Layer (first hidden layer)
- Code layer (our middle layer that will be our "coded" representation)
- Decoder Hidden Layer (same size as Encoder Hidden Layer)
- Output Layer (same size as Input Layer)

To start, we're just going to train a vanilla autoencoder. This will learn whatever weights the optimizer happens to settle on after training. 

In later sections we'll see how to constrain the weights learned based on useful properties.

In [ ]:
input_size = 784  # 784 = 28*28 (images are 28 by 28 pixels... grayscaled)
hidden_size = 128  # We're using  single hidden layer. 
code_size = 32  # Dimension of our "coded" or compressed representation. 32 is much less than the 784 input dimension.

# This is how you construct your network in Keras. Chaining the layers by referencing the previous layer.
# Really damn cool and simple.
input_img = Input(shape=(input_size,))
hidden_1 = Dense(hidden_size, activation='relu')(input_img)
code = Dense(code_size, activation='relu')(hidden_1)
hidden_2 = Dense(hidden_size, activation='relu')(code)
output_img = Dense(input_size, activation='sigmoid')(hidden_2)

# With our layers defined, then the full ANN model is created by specifying the input and output.
autoencoder = Model(input_img, output_img)
# Define our optimizer and loss function
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
# And train it. 
# Automatically uses multiple cores (if you have them). Can also use GPUs if you've got them.
autoencoder.fit(x_train, x_train, epochs=3)

# Interpretation
## Accuracy
With our model trained, let's compare some inputs to outputs.

In [ ]:
# Pass in the learned model, the image size (must be same used for the input layer size reshaped to be 2-d), 
# and the number of images to show (default is 5)
# Note that the held out test set x_test is used.
plot_autoencoder_outputs(autoencoder, (28, 28), 6)

Pretty good considering that the inputs are 784 dimensional vectors and we're compressing them down to a 32 dimensional representation.

## Feature Construction
This may be ill-named, but what we can do with our learned model is see what the learned weights are doing to the input vectors at the first layer. 

This can give us some insight into what the learned network is doing to the input images to "best" represent them so that the 32-dimensional code layer is able to efficiently reconstruct the outputs accurately.

If we focus on the weights connecting the first layer (input image that's flattened from 28x28 to 784) to the next layer (128 nodes), then we have a 784 by 128 matrix:

In [ ]:
autoencoder.get_weights()[0].shape

In [ ]:
weights = autoencoder.get_weights()[0].T

n = 10  # Just plotting the first 10 hidden layer nodes.
plt.figure(figsize=(20, 5))
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.imshow(weights[i+0].reshape(28, 28))  # Get the weights going to hidden layer node i. Reshape.
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    

# Sparse Autoencoders
Okay. This is really cool. 

We're going to use a shallower network. Just
- Input Layer
- Code Layer
- Output Layer

And, we're going to compare a vanilla learned network (just like we did above, but with less layers) against a network learned with a penalty for having non-zero weights. 

This is an example of using a regularizer. With Keras, this is super easy. Just add an additional argument to the specification of the Code Layer:
```
code = Dense(code_size, activation='relu', activity_regularizer=l1(10e-6))(input_img)
```
The `l1(10e-6)` isn't terribly clear to me. But, according to https://towardsdatascience.com/applied-deep-learning-part-3-autoencoders-1c083af4d798 it's usually a value between `0.001` and `0.000001`. I suspect choosing this is like choosing your learning rate. 

In [ ]:
input_size = 784
code_size = 32

input_img = Input(shape=(input_size,))
code = Dense(code_size, activation='relu')(input_img)
output_img = Dense(input_size, activation='sigmoid')(code)

autoencoder_standard = Model(input_img, output_img)
autoencoder_standard.compile(optimizer='adam', loss='binary_crossentropy')
history_standard = autoencoder_standard.fit(x_train, x_train, epochs=20)

encoded_standard = Model(input_img, code)

In [ ]:
input_size = 784
code_size = 32

input_img = Input(shape=(input_size,))
code = Dense(code_size, activation='relu', activity_regularizer=l1(10e-6))(input_img)  # Note the regularizer
output_img = Dense(input_size, activation='sigmoid')(code)

autoencoder_regularized = Model(input_img, output_img)
autoencoder_regularized.compile(optimizer='adam', loss='binary_crossentropy')
history_regularized = autoencoder_regularized.fit(x_train, x_train, epochs=20)

encoded_regularized = Model(input_img, code)

In [ ]:
# Plot the regularized model reconstruction
plot_autoencoder_outputs(autoencoder_regularized, (28, 28), 7)

Really good still! And we can compare the lsos functions of vanilla and regularized.

In [ ]:
# Compare the loss functions. Note that since this is an auto-encoder, we only have the loss variable to plot. No accuracy.
plot_compare_histories([history_standard, history_regularized], 
                       ['Standard Autoencoder', 'Regularized Autoencoder'], plot_accuracy=False)

## Regularized Model Structure
How can we understand or verify that the regularized model is indeed more sparse? 

To start, we can have the models predict the "coded" layer on the held out test set `x_test`. This gives a 32-dimensional vector for each image in the test set.

In [ ]:
# Calcualte the mean of each 32 dimensional predicted "code" across the test set
encoded_standard.predict(x_test).mean(axis=1)

In [ ]:
# Same for the regularized model.
encoded_regularized.predict(x_test).mean(axis=1)

Not terribly easy to interpret the above with the output truncated. We can take the mean() acrosss the 32 dimensions too, to get a single number that communicates the average value of the 32 dimensional coded representations of the test set.

In [ ]:
print(encoded_standard.predict(x_test).mean())
print(encoded_regularized.predict(x_test).mean())

Not a big difference, but something. 

Also, each element is `>=0`. I don't entirely understand why there are no negative values. Maybe something to do with the loss function and inputs only being non-negative?

Lastly, let's plot the distribution of the predicted values. This is a more meaningful representation of the "coded" layer, compared to just reporting the means.

In [ ]:
standard_scores = encoded_standard.predict(x_test).ravel()  # ravel() flattens the matrix of 1000 by 32
regularized_scores = encoded_regularized.predict(x_test).ravel()
sns.distplot(standard_scores, hist=False, label='standard model')
ax = sns.distplot(regularized_scores, hist=False, label='regularized model')
ax.set(xlabel='Coded Vector Value', ylabel='Density')
plt.legend()
plt.show()

You can see that the regularized model has a higher density at lower values than the standard/vanilla one. That aligns with our expectations that the regularized model has more values at/near 0.

## Compare Weights
Last thing I'd like to do is compare the weights visualization of the standard vs regularized. 

We've got a 32 node coded layer so going to plot them all.

In [ ]:
weights = encoded_standard.get_weights()[0].T

n = 32  # Just plotting the first 10 hidden layer nodes.
plt.figure(figsize=(20, 5))
for i in range(n):
    ax = plt.subplot(4, n/4, i + 1)
    plt.imshow(weights[i+0].reshape(28, 28))  # Get the weights going to hidden layer node i. Reshape.
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

In [ ]:
weights = encoded_regularized.get_weights()[0].T

n = 32  # Just plotting the first 10 hidden layer nodes.
plt.figure(figsize=(20, 5))
for i in range(n):
    ax = plt.subplot(4, n/4, i + 1)
    plt.imshow(weights[i+0].reshape(28, 28))  # Get the weights going to hidden layer node i. Reshape.
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

Nothing really stands out. Some of them seem to indicate structure, like a ring, but both regularized and standard show it.